In [ ]:
# ALPHA VERSION 11.1 ==========================================================
# Apply market indicators and basic commonality analysis
# to detect bottom and trigger via Telegram & Socket.
# Then Create new database on MariaDB
# =============================================================================
# Library for process Datetime
from datetime import datetime, timedelta
import time
import pytz
# Library for Data Analysis
import pandas as pd
# Library for POST GET & Socket requests
import requests
import urllib.parse
import json
import socket
# Connect to mariaDB to perform CRUD
import mariadb

# Pre-define and CORE Function ================================================
# Database connection parameters
conn_params= {
    # "host" : "103.173.66.235",
    "host" : "localhost",
    "database" : "crypto",
    "user" : "root",
    "password" : "Trien825590"
}
# Define the list of 40 interesting coins
tuple_coin = (
  "UNIVNDC","CETUSVNDC","DOGEVNDC","JASMYVNDC","SOLVNDC",
  "MCAKEVNDC","SXPVNDC","CKBVNDC","EGPT1000VNDC","UXLINKVNDC",
  "MIAVNDC","MOODENGVNDC","EDU3VNDC","MEON1000VNDC","DOGSVNDC",
  "SUIVNDC","FOXYVNDC","BIGTIMEVNDC","TRUVNDC","OPVNDC",
  "LINKVNDC","XRPVNDC","1000CATVNDC","WIFVNDC","NEIROVNDC",
  "XLMVNDC","KSMVNDC","DOTVNDC","ENSVNDC","SANDVNDC",
  "GRASSVNDC","LYNXVNDC","AVAXVNDC","LISTAVNDC","TIAVNDC",
  "PEPE1000VNDC","WLDVNDC", "ADAVNDC", "HBARVNDC", "LDOVNDC"
)

# Define the GMT+7 timezone
timezone = pytz.timezone('Asia/Bangkok')
# Store last trigger timestamp, delay to avoid spam by continuous triggering
last_trigger = [0.0] * len(tuple_coin)
# Set the interval for the status message (in seconds)
interval = 60*60  # 1 hours in seconds
# Check point of important handling - in UNIX timestamp
check_point = time.time()

# Define your quiet hours not to send Telegram
quiet_hours = [
    ((22, 0),(23, 59)),  # 10 PM to midnight
    ((0, 0),(6, 0))      # Midnight to 6 AM
]

# Function to send a message to Telegram
def send_telegram(message_string, max_retry=2):
    # Telegram bot token and chat ID
    TOKEN = "5614737400:AAHbvZrJbomt09EkpPuhadBCJl7NaGu6rlg"
    ID = "5559031253"

    # Check if the time now is quiet hour or not
    current_time = datetime.now(timezone)
    for start, end in quiet_hours:
        quiet_start = current_time.replace(hour=start[0], minute=start[1], second=0, microsecond=0)
        quiet_end = current_time.replace(hour=end[0], minute=end[1], second=59, microsecond=0)
        if quiet_start <= current_time <= quiet_end:
            return False
    # Then continue process send to Telegram
    current_retry = 0
    while (current_retry <= max_retry):
        date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
        if current_retry > 0:
            print(f"{date_time} Telegram: Retry #{current_retry}")
            time.sleep(5)   # Wait a second to stabilize the server
        try:
            # URL encode the message string
            encoded_message = urllib.parse.quote(message_string)
            # Construct the URL
            url = f"https://api.telegram.org/bot{TOKEN}/sendMessage?chat_id={ID}&text={encoded_message}"
            # Send the request
            response = requests.post(url, timeout=5)
            response.raise_for_status()
            return True
        except requests.exceptions.RequestException as e:            
            print(date_time, f"ERROR send Telegram: \n{e}")
            current_retry += 1
            if current_retry > max_retry:
                print(date_time, "TELEGRAM: Max retries reached.")
                return False
        except requests.exceptions.HTTPError as http_err:
            print(f"Telegram HTTP error occurred: {http_err}")  # Handle the HTTP error
        except Exception as err:
            print(f"Telegram other error occurred: {err}")      # Handle any other errors

# Function get price 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 12h, 1d
def get_data(coin_name, time_unit, time_range, max_retry=5, time_out=10):
    current_retry = 0
    url = "https://api-pro.goonus.io/perpetual/v1/klines"
    # Get the current Unix timestamp value in ms
    current_time = int(time.time()) * 1000
    # Define the parameters for the API request
    params = {
        "symbol": coin_name,
        "interval": time_unit,
        "endTime": current_time,
        # "limit": "600"
    }
    headers = {
        "Content-Type": "application/json",
        "Connection": "keep-alive"
    }
    while current_retry <= max_retry:
        if current_retry > 0:
            print(f"GET Crypto {coin_name}: Retry #{current_retry}")
            time.sleep(5)   # Wait a second to stabilize the server
        try:
            # Try request & get the Response during 5 seconds
            response = requests.get(url, params=params, headers=headers, timeout=time_out)
            # Raise an HTTPError for bad responses (4xx and 5xx)
            response.raise_for_status()

            historical_data = response.json()
            # Map the time_range matching with the total record
            while len(historical_data) < time_range:
                time_range -= 10
            # Check if the data has sufficient 100 records
            if len(historical_data) >= 100:
                # Take the latest "time_range" elements in the list
                historical_data = historical_data[-time_range:]
                # [Time,Open,High,Low,Close,Base qty,Quote qty,Time]
                # Filter to only include 'timestamp' and 'close'
                filtered_data = [[
                    # Convert Unix timestamp -> GMT +7 hours (+25200)
                    datetime.fromtimestamp(int(item[0])/1000, tz=pytz.utc).astimezone(timezone).strftime('%Y-%m-%d %H:%M:%S'),
                    # Close price
                    float(item[4]),
                    # Percentage difference
                    round(100*(float(item[2])-float(item[3]))/float(item[1]),2),
                    float(item[2]), float(item[3])
                ] for item in historical_data]
                # [timestamp, close, %price_diff, Highest, Lowest]
                # print(filtered_data)
                return filtered_data
            else:
                date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
                message = f"Error get {coin_name} - Not enough data ({len(historical_data)})"
                print(date_time, message)
                send_telegram(message)
                return None
        except requests.exceptions.RequestException as e:
            date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
            print(date_time, f"ERROR Request {coin_name} - retry #{current_retry}\n{e}")
            current_retry += 1
            if current_retry > max_retry:
                print(f"{date_time}: Max retries reached. Raising exception.")
                send_telegram(f"ERROR get: {coin_name} - Max retries\n{e}")
                return None
        except requests.exceptions.HTTPError as http_err:
            print(f"Get crypto HTTP error occurred: {http_err}")  # Handle the HTTP error
        except Exception as err:
            print(f"Get crypto other error occurred: {err}")      # Handle any other errors

# Function call API to create in mySQL database ===============================
def create_database(coin_name, test_id, date_trigger, trigger_value, trigger_note, max_retry=2):
    current_retry = 0
    while current_retry <= max_retry:
        date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
        if current_retry > 0:
            print(f"CREATE DB Retry #{current_retry}")
            time.sleep(5)

        # Create connection & implement cursor
        connection = mariadb.connect(**conn_params)
        cursor = connection.cursor()

        try:
            
            # Check if duplicate
            query = "SELECT COUNT(*) FROM `crypto_backtest` WHERE coin_name = ? AND date_trigger = ?"
            data = (coin_name, date_trigger)
            cursor.execute(query,data)
            if cursor.fetchone()[0] > 0:
                print(date_time, "CREATE DB: Duplicate record found")
                # Close Connection & Free resources
                cursor.close()
                connection.close()
                return False
            # Execute the CREATE new record
            query = "INSERT INTO crypto_backtest (coin_name, test_id, date_trigger,trigger_value, trigger_note) VALUES (?,?,?,?,?)"
            data = (coin_name, test_id, date_trigger, trigger_value, trigger_note)
            cursor.execute(query, data)
            connection.commit()
            # Close Connection & Free resources
            cursor.close()
            connection.close()
            # Logging
            print(date_time, "Success create new DB record")
            return True
        except Exception as e:
            print(f"{date_time}: CREATE DB failed\n{e}")
            send_telegram(f"CREATE DB failed #{current_retry}\n{e}")
            current_retry += 1
            if current_retry > max_retry:
                print(f"{date_time}: CREATE DB Max retries reached.")
                send_telegram(f"CREATE DB: Max retries reached.")
                # Close Connection & Free resources
                cursor.close()
                connection.close()
                return False

# Function Send Order via socket ==============================================
def trigger_socket(coin_name, order_price, variance):
    # Make JSON data payload
    data_send = {
        "coin_name": coin_name,
        "order_price": order_price,
        "variance": variance
    }
    date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.connect(('localhost', 65432))
            s.sendall(json.dumps(data_send).encode('utf-8'))
    except ConnectionRefusedError:
        print(date_time, "Error Socket: Connection refused")

# MARKET INDICATOR Function ===================================================
# Function to calculate the Simple Moving Average (SMA)
def calculate_sma(data, window=5):
    if len(data) < window:
        print("Not enough data to calculate SMA")
        send_telegram("Not enough data to calculate SMA")
        return pd.Series([None] * len(data))
    return data['close'].rolling(window=window).mean()

# Function to calculate the Relative Strength Index (RSI)
def calculate_rsi(data, window=15):
    if len(data) < window:
        print("Not enough data to calculate RSI")
        send_telegram("Not enough data to calculate RSI")
        return pd.Series([None] * len(data))

    delta = data['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

# MAIN PROGRAM ================================================================
if __name__ == '__main__':
    # Send log to Telegram
    date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
    message = f"{date_time}: Alpha started"
    print(message)
    send_telegram(message)
    
    # Infinite loop to execute
    while True:
        # Example for fetching historical data for one-by-one coin
        # for i in range(1):
        for i in range(len(tuple_coin)):
            # Collect the short-term coin data (1 min)
            datacoin_short = get_data(tuple_coin[i], "1m", 600)
            # time.sleep(1)  # Sleep 1s for stablilize the server request

            if datacoin_short is None:
                continue    # Pass to the next loop

            # Convert to DataFrame for further process
            df = pd.DataFrame(datacoin_short, columns=['timestamp', 'close', 'percent', 'high', 'low'])
            # Format correct data type
            df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
            df['percent'] = df['percent'].astype(float)
            df['close'] = df['close'].astype(float)
            df['high'] = df['high'].astype(float)
            df['low'] = df['low'].astype(float)
            # Calculate average number of 'percent' last 30 records
            df['variance'] = df['percent'].rolling(window=30).mean()

            # Measure the Statistical Indicators =============================
            df['SMA_5'] = calculate_sma(df, 6)
            # Calculate the percentage difference
            # df['SMA_5_diff'] = df['SMA_5'].diff()
            df['SMA_5_diff'] = df['SMA_5'].diff() / df['SMA_5'].shift(1) * 100
            df['SMA_25'] = calculate_sma(df, 25)
            df['RSI_15'] = calculate_rsi(df)
            df['diff'] = 100 * (df['SMA_5'] - df['SMA_25']) / df['close']
            # Set the timestamp as the index
            # df.set_index('timestamp', inplace=True)
            # display(df)

            # Implement the swing strategy: buy the lowest
            average_before = df['SMA_5_diff'].iloc[-8:-3].mean()
            average_after = df['SMA_5_diff'].tail(3).mean()
            multiple = average_after * average_before

            # ALPHA Condition =================================================
            if  (df.iloc[-1]['diff'] <= -0.3) and\
                (df.iloc[-1]['RSI_15'] < 30) and\
                ((time.time() - last_trigger[i]) >= 900) and\
                (df.iloc[-1]['variance'] > 0.1) and\
                (average_before < 0) and (multiple < 0):

                message = f"""	SMA5: {round(df.iloc[-1]['diff'], 3)} 
	SMA25: {round(df.iloc[-1]['SMA_25'], 3)} 
	Diff: {round(df.iloc[-1]['diff'], 3)} 
	RSI15: {round(df.iloc[-1]['RSI_15'], 1)} 
	Variance: {round(df.iloc[-1]['variance'], 3)} 
	Before: {round(average_before, 3)} 
	After: {round(average_after, 3)}"""

                date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
                print(date_time, tuple_coin[i]+"\n"+ message)
                send_telegram(tuple_coin[i] + "\n" + message)
                last_trigger[i] = time.time()
                # Create new Record in Back Testing database
                create_database(tuple_coin[i], "BT08", date_time, df.iloc[-1]['close'].astype(str), message)
                # Trigger signal via socket to place order
                trigger_socket(tuple_coin[i], df.iloc[-1]['close'],df.iloc[-1]['variance'])

        current_time = time.time()
        if (current_time - check_point) >= interval:
            date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
            check_point = current_time
            # Process Back Testing
            send_telegram("Alpha still working")

        time.sleep(5)

2024-12-07 22:46:50: Alpha started
2024-12-07 22:47:00 ERROR Request UNIVNDC - retry #0
HTTPSConnectionPool(host='api-pro.goonus.io', port=443): Read timed out. (read timeout=10)
GET Crypto UNIVNDC: Retry #1
2024-12-07 22:47:15 ERROR Request UNIVNDC - retry #1
HTTPSConnectionPool(host='api-pro.goonus.io', port=443): Read timed out. (read timeout=10)
GET Crypto UNIVNDC: Retry #2
2024-12-07 22:47:30 ERROR Request UNIVNDC - retry #2
HTTPSConnectionPool(host='api-pro.goonus.io', port=443): Read timed out. (read timeout=10)
GET Crypto UNIVNDC: Retry #3
2024-12-07 22:47:45 ERROR Request UNIVNDC - retry #3
HTTPSConnectionPool(host='api-pro.goonus.io', port=443): Read timed out. (read timeout=10)
GET Crypto UNIVNDC: Retry #4
2024-12-07 22:48:00 ERROR Request UNIVNDC - retry #4
HTTPSConnectionPool(host='api-pro.goonus.io', port=443): Read timed out. (read timeout=10)
GET Crypto UNIVNDC: Retry #5


KeyboardInterrupt: 

In [12]:
import mariadb
# Library for process Datetime
from datetime import datetime, timedelta
import time
import pytz

# Define the GMT+7 timezone
timezone = pytz.timezone('Asia/Bangkok')

date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
# connection parameters
conn_params= {
    "user" : "root",
    "password" : "Trien825590",
    "host" : "103.173.66.235",
    "database" : "crypto"
}

# Establish a connection
connection= mariadb.connect(**conn_params)

cursor= connection.cursor()


# # # Populate countries table  with some data
# cursor.execute("INSERT INTO crypto_backtest (coin_name, test_id, date_trigger,trigger_value, trigger_note) VALUES (?,?,?,?,?)",
#     ("UNIVNDC", "BT08", date_time, 234234.341, "Teset nha các bạn trẻ trâu"))
# connection.commit()

# # # # Populate countries table  with some data
# # cursor.execute("INSERT INTO crypto_backtest (coin_name, test_id, date_trigger, trigger_note) VALUES (?,?,?)",
# #     ("BTCVNDC", "BT09", "Berlin"))
# # cursor.commit()

# # retrieve data
# cursor.execute("SELECT * FROM crypto_backtest")

# # print content
# # row = cursor.fetchone()
# rows = cursor.fetchall()
# print(rows)

# cursor.execute("SELECT * FROM crypto_backtest")
# for i in cursor:
#     print(f"Successfully retrieved {i}")
query = "SELECT COUNT(*) FROM `crypto_backtest` WHERE coin_name = ? AND date_trigger = ?"
data = ("BTCVNDC","2024-12-07 22:30:54")
cursor.execute(query,data)
print(cursor.fetchone()[0])
# free resources
cursor.close()
connection.close()

0


In [ ]:
import mariadb

# connection parameters
conn_params= {
    "user" : "example_user",
    "password" : "GHbe_Su3B8",
    "host" : "localhost",
    "database" : "test"
}

# Establish a connection
connection= mariadb.connect(**conn_params)

cursor= connection.cursor()
sql= "INSERT INTO countries (name, country_code, capital) VALUES (?,?,?)"

data= [("Ireland", "IE", "Dublin"),
       ("Italy", "IT", "Rome"),
       ("Malaysia", "MY", "Kuala Lumpur"),
       ("France", "FR", "Paris"),
       ("Iceland", "IS", "Reykjavik"),
       ("Nepal", "NP", "Kathmandu")]

# insert data
cursor.executemany(sql, data)

# Since autocommit is off by default, we need to commit last transaction
connection.commit()

# Instead of 3 letter country-code, we inserted 2 letter country code, so
# let's fix this mistake by updating data
sql= "UPDATE countries SET country_code=? WHERE name=?"
data= [("Ireland", "IRL"),
       ("Italy", "ITA"),
       ("Malaysia", "MYS"),
       ("France", "FRA"),
       ("Iceland", "ISL"),
       ("Nepal", "NPL")]
cursor.executemany(sql, data)

# Now let's delete all non European countries
sql= "DELETE FROM countries WHERE name=?"
data= [("Malaysia",), ("Nepal",)]
cursor.executemany(sql, data)

# by default autocommit is off, so we need to commit
# our transactions
connection.commit()

# free resources
cursor.close()
connection.close()

In [ ]:
SELECT COUNT(*) FROM `crypto_backtest` WHERE coin_name = ? AND date_trigger = ?"